In [68]:

from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.problems.single import Rastrigin
from pymoo.optimize import minimize
from pymoo.indicators.gd import GD
from pymoo.indicators.igd import IGD

problem = get_problem("Rastrigin")

algorithm = PSO()

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

pf = get_problem("Rastrigin").pareto_front()

ind = GD(pf)
print("GD", ind(res.X))

ind2 = IGD(pf)
print("IGD", ind2(res.X))

Best solution found: 
X = [6.75220419e-07 1.76711118e-07]
F = [9.66480229e-11]
GD 6.979609109362503e-07
IGD 6.979609109362503e-07


In [70]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.problems import get_problem
from pymoo.optimize import minimize

problem = get_problem("Rastrigin")

algorithm = GA(
    pop_size=9000,
    eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))
pf = get_problem("Rastrigin").pareto_front()

ind = GD(pf)
print("GD", ind(res.X))

ind2 = IGD(pf)
print("IGD", ind2(res.X))

Best solution found: 
X = [-0.0017443   0.00046954]
F = [0.00064736]
GD 0.0018063910168840361
IGD 0.0018063910168840361


In [69]:
import numpy as np

from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.unsga3 import UNSGA3
from pymoo.problems import get_problem
from pymoo.optimize import minimize

problem = get_problem("Rastrigin")

# create the reference directions to be used for the optimization - just a single one here
ref_dirs = np.array([[1.0]])

# create the algorithm object
algorithm = UNSGA3(ref_dirs, pop_size=100)

# execute the optimization
res = minimize(problem,
               algorithm,
               termination=('n_gen', 150),
               save_history=True,
               seed=1)

print("UNSGA3: Best solution found: \nX = %s\nF = %s" % (res.X, res.F))
pf = get_problem("Rastrigin").pareto_front()

ind = GD(pf)
print("GD", ind(res.X))

ind2 = IGD(pf)
print("IGD", ind2(res.X))

UNSGA3: Best solution found: 
X = [-6.38345175e-06 -1.12349942e-05]
F = [3.31262271e-08]
GD 1.2921824565198275e-05
IGD 1.2921824565198275e-05
